<a href="https://colab.research.google.com/github/AmaleshV/Medium-Posts/blob/master/Automate_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Increase **RAM**

In [0]:
d=[]
while(1):
  d.append('1')

# Check GPU 

In [0]:
!nvidia-smi

Tue Jan  7 09:29:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Kaggle Pipeline

**Upload Kaggle JSON**

In [0]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


**Downloading the datasets from API calls**

In [0]:
%%time
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download --force ieee-fraud-detection
!unzip sample_submission.csv.zip
!unzip test_identity.csv.zip 
!unzip test_transaction.csv.zip
!unzip train_identity.csv.zip
!unzip train_transaction.csv.zip

 81% 47.0M/58.3M [00:03<00:01, 7.57MB/s]
100% 58.3M/58.3M [00:03<00:00, 17.4MB/s]
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 106MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 160MB/s]
  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 109MB/s]
 94% 49.0M/52.2M [00:03<00:00, 11.2MB/s]
100% 52.2M/52.2M [00:03<00:00, 16.0MB/s]
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  test_identity.csv.zip
  inflating: test_identity.csv       
Archive:  test_transaction.csv.zip
  inflating: test_transaction.csv    
Archive:  train_identity.csv.zip
  inflating: train_identity.csv      
Archive:  train_transaction.csv.zip
  inflating: train_transaction.csv   
CPU times: user 301 ms, sys: 55 ms, total: 356 ms
Wall time: 50.6 s


**Import Packages**

In [0]:
!pip install xgboost
import xgboost as xgb
import pandas as pd
import numpy as np
import random
import sklearn
import re

**Pre-Processing and Data Wrangling**

In [0]:
%%time
import dask.dataframe as dd
train_identity = dd.read_csv('train_identity.csv')
train_transaction = dd.read_csv('train_transaction.csv')
test_identity = dd.read_csv('test_identity.csv')
test_transaction = dd.read_csv('test_transaction.csv')
sub = dd.read_csv('sample_submission.csv')
# let's combine the data and work with the whole dataset
train = dd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = dd.merge(test_transaction, test_identity, on='TransactionID', how='left')

CPU times: user 751 ms, sys: 25.7 ms, total: 776 ms
Wall time: 860 ms


In [0]:
#convert to pandas
train = train.compute()
test = test.compute()

In [0]:
#Data Wrangling
df=train
df=df.sample(frac=1,random_state=123) #shuffle data for memory bias
#from sklearn import preprocessing
# Identify which variables can be numeric and which factor (<32 uniq levels)
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
cat_cols= [col for col in df.columns if (df[col].dtypes==object) & (df[col].nunique() > 32)]
print(cat_cols)
df= df.drop(cat_cols, axis=1)
#df= df.loc[:,(df.dtypes== 'int64') | (df.dtypes==float) |((df.dtypes==object) & (df.nunique() < 32))]
# Select only columns with less than 32 unique values if object or select numeric variables
df_wrangled=df
df_wrangled=train
if 'isFraud' in df_wrangled:
        df_wrangled = df_wrangled.loc[df_wrangled['isFraud'].notnull(),:] #pick only those alerts with target variable defined
print(df_wrangled.head())
# # Train Test Split
# set x and y variables
if 'isFraud' in df_wrangled:
    y = df.isFraud
    x= df.loc[:, df.columns != 'isFraud']
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.1, random_state = 123)

['P_emaildomain', 'R_emaildomain', 'id_30', 'id_31', 'id_33', 'DeviceInfo']
   TransactionID  isFraud  ...  DeviceType                     DeviceInfo
0        2987000        0  ...         NaN                            NaN
1        2987001        0  ...         NaN                            NaN
2        2987002        0  ...         NaN                            NaN
3        2987003        0  ...         NaN                            NaN
4        2987004        0  ...      mobile  SAMSUNG SM-G892A Build/NRD90M

[5 rows x 434 columns]


In [0]:
# # Training Data preprocessing
# Eliminate automatically variables with more than 20% of missingness
xTrain = xTrain[xTrain.columns[xTrain.isnull().mean() < 0.2]]
# fill missing values with mean column values for numeric features
#xTrain.select_dtypes(include=numerics) =  xTrain.select_dtypes(include=numerics).fillna(xTrain.mean(), inplace=True)
#xTrain._get_numeric_data().fillna(xTrain.mean(), inplace=True)
cols= xTrain.columns
colnames_numerics_only = xTrain.select_dtypes(include=np.number).columns.tolist()
colnames_numerics_only
xTrain[colnames_numerics_only] = xTrain[colnames_numerics_only].fillna(xTrain[colnames_numerics_only].mean())
# impute categorical variables with mode
#cols = xTrain.select_dtypes(include= ).columns
cat_cols= list(set(cols) - set(colnames_numerics_only))
#xTrain[cat_cols] = xTrain.loc[:,xTrain.columns.isin(cat_cols) & (xTrain.isna().any())].fillna(xTrain[cat_cols].mode().iloc[0])
xTrain[cat_cols] = xTrain[cat_cols].fillna(xTrain[cat_cols].mode().iloc[0])
train_cols = xTrain.columns
#xTrain.loc[:,(xTrain.dtypes==object)] = xTrain.loc[:,(xTrain.dtypes==object)].fillna(xTrain.mode())
# get categorical features index
xTrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531486 entries, 36908 to 19979
Columns: 180 entries, TransactionID to V321
dtypes: float64(174), int64(3), object(3)
memory usage: 733.9+ MB


**Feature Selection**

In [0]:
%%time
xTrain_dummy = pd.get_dummies(xTrain, prefix_sep='_', drop_first=True) #create dummies
xgb_cols =xTrain_dummy.columns
xTrain_xgb=xTrain_dummy
yTrain_xgb = yTrain
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier
xgbc=XGBClassifier(n_estimators=500,verbose=1,tree_method='gpu_hist',random_state=123)
embeded_xgb_selector = SelectFromModel(xgbc)
embeded_xgb_selector.fit(xTrain_xgb, yTrain_xgb)
embeded_xgb_support = embeded_xgb_selector.get_support()
embeded_xgb_feature = xTrain_xgb.loc[:,embeded_xgb_support].columns.tolist()
final_selected_cols=embeded_xgb_feature
final_tr=xTrain_xgb[embeded_xgb_feature]

CPU times: user 8.33 s, sys: 3.72 s, total: 12 s
Wall time: 12 s


**Model Building**

In [0]:
%%time
import xgboost as xgb
# Parameter Tuning maxd=50,minc=1
model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=10,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=123,
    tree_method='gpu_hist') #after hyperparameterisation
xgb_best=model.fit(final_tr, yTrain)

CPU times: user 10.8 s, sys: 3.24 s, total: 14 s
Wall time: 14 s


**Model Validation**

In [0]:
### Test Data preprocessing ###
# Eliminate automatically variables with more than 20% of missingness
xTest = xTest[xTest.columns[xTest.isnull().mean() < 0.2]]
# fill missing values with mean column values for numeric features
test_cols= xTest.columns
num_cols = xTest.select_dtypes(include=np.number).columns
xTest[num_cols] = xTest[num_cols].fillna(xTest[num_cols].mean())
# impute categorical variables with mode
#cols = xTest.select_dtypes(include= ).columns
cat_cols= list(set(test_cols) - set(num_cols))
#xTest[cat_cols] = xTest.loc[:,xTest.columns.isin(cat_cols) & (xTest.isna().any())].fillna(xTest[cat_cols].mode().iloc[0])
xTest.loc[:,cat_cols] = xTest[cat_cols].fillna(xTest[cat_cols].mode().iloc[0])
# Select only those features which are there in training #
xTest = xTest[train_cols] 
# Dummify categorical vars
xTest_dummy = pd.get_dummies(xTest, prefix_sep='_', drop_first=True)

##missing columns levels train and test
missing_levels_cols= list(set(xTrain_dummy.columns) - set(xTest_dummy.columns))
for c in missing_levels_cols:
    xTest_dummy[c]=0
# Select only those variables in final_tr
final_ts =xTest_dummy[final_selected_cols]
xTest.columns
##test prediction
ytest_xgb = xgb_best.predict_proba(final_ts)
### Performance metrics on test data ###
from sklearn.metrics import roc_curve,roc_auc_score,f1_score,cohen_kappa_score,precision_score,recall_score,confusion_matrix
yTest_num= yTest
roc_auc_score(yTest_num,ytest_xgb[:,1])

0.9313639465901744

**Submission File**

In [0]:
####submission prediction
### Test Data preprocessing ###
# Eliminate automatically variables with more than 20% of missingness
test = test[test.columns[test.isnull().mean() < 0.2]]
# fill missing values with mean column values for numeric features
test_cols= test.columns
num_cols = test.select_dtypes(include=np.number).columns
test[num_cols] = test[num_cols].fillna(test[num_cols].mean())

# impute categorical variables with mode
#cols = .select_dtypes(include= ).columns
cat_cols= list(set(test_cols) - set(num_cols))
#test[cat_cols] = test.loc[:,test.columns.isin(cat_cols) & (test.isna().any())].fillna(test[cat_cols].mode().iloc[0])
test.loc[:,cat_cols] = test[cat_cols].fillna(test[cat_cols].mode().iloc[0])

# Select only those features which are there in training #
test = test[train_cols] 

# Dummify categorical vars
test_dummy = pd.get_dummies(test, prefix_sep='_', drop_first=True)

##missing columns levels train and test
missing_levels_cols= list(set(xTrain_dummy.columns) - set(test_dummy.columns))

for c in missing_levels_cols:
    test_dummy[c]=0

# Select only those variables in final_tr
final_ts =test_dummy[final_selected_cols]
test.columns
#ytest_xgb1 = xgb_best.predict_proba(final_ts)
ytest_xgbs=xgb_best.predict_proba(final_ts)
### Performance metrics on test data ###
submission_xgb= np.c_[test.TransactionID,ytest_xgbs[:,1]]
submission_xgb= pd.DataFrame(submission_xgb)
submission_xgb.columns= ['TransactionID','isFraud']
print(submission_xgb)
submission_xgb.to_csv('submission_xgb.csv')
from google.colab import files
files.download('submission_xgb.csv')